# Problema 4.1

Selecao de Atividades de uma Fábrica 

## Enunciado


Recursos: 
- Materia prima 1
- Materia prima 2
- Mão de obra e equipamentos

Atividades:
- Fabricar produto 1
- Fabricar produto 2
- Fabricar produto 3
- Fabricar produto 4


In [41]:
# Quantidade necessária da materia prima i (mpi) para produzir 1 unidade do produto j
custos = {
    "mp1": [1, 5, 10, 2],
    "mp2": [4, 1, 3, 8],
}

In [42]:
# Quantidade disponível da materia prima i
materia_prima_disponivel = {
    "mp1": 80_000,
    "mp2": 100_000,
}

In [43]:
# Mao de obra e equipamentos disponíveis permitem fabricar:
# - 12000 unidades do produto 1 OU
# - 15000 unidades do produto 2 OU
# - 20000 unidades do produto 3 OU
# - 10000 unidades do produto 4.

In [44]:
# lucro unitário por produto pj
lucro = {
    "p1": 4,
    "p2": 3,
    "p3": 8,
    "p4": 6,
}

## Modelagem

In [45]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver("GLOP")
if not solver:
    print("Solver nao encontrado")

### Variáveis de decisão
- $x_{1}$: quantidade de produto 1 a ser produzida
- $x_{2}$: quantidade de produto 2 a ser produzida
- $x_{3}$: quantidade de produto 3 a ser produzida
- $x_{4}$: quantidade de produto 4 a ser produzida

In [46]:
x1 = solver.NumVar(0, solver.infinity(), "x1")
x2 = solver.NumVar(0, solver.infinity(), "x2")
x3 = solver.NumVar(0, solver.infinity(), "x3")
x4 = solver.NumVar(0, solver.infinity(), "x4")

decision_vars = [x1, x2, x3, x4]

## Função objetivo

In [47]:
# Queremos maximizar o lucro total
# lucro total = (lucro unitário) * (quantidade produzida)
solver.Maximize(
    lucro["p1"] * x1 + lucro["p2"] * x2 + lucro["p3"] * x3 + lucro["p4"] * x4
)

## Restrições

In [48]:
# eq 1. quantidade de materia prima 1 disponivel
solver.Add(
    x1 * custos["mp1"][0]
    + x2 * custos["mp1"][1]
    + x3 * custos["mp1"][2]
    + x4 * custos["mp1"][3]
    <= materia_prima_disponivel["mp1"],
    name="constraint1",
)

# eq 2. quantidade de materia prima 2 disponivel
solver.Add(
    x1 * custos["mp2"][0]
    + x2 * custos["mp2"][1]
    + x3 * custos["mp2"][2]
    + x4 * custos["mp2"][3]
    <= materia_prima_disponivel["mp2"],
    name="constraint2",
)


# eq 3. limite de mao de obra para os produto 1, 2, 3 e 4:
# x1 / 12000 + x2 / 15000 + x3 / 20000 + x4 / 10000 <= 1
# x1 / 12000 + x2 / 15000 + x3 / 20000 + x4 / 10000 - 1 <= 0
solver.Add(x1 / 12000 + x2 / 15000 + x3 / 20000 + x4 / 10000 <= 1, name="constraint3")

# eq 4. nao produzir quantidade negativa de produtos
# x1 >= 0
# x2 >= 0
# x3 >= 0
# x4 >= 0
solver.Add(x1 >= 0, name="constraint4.1")
solver.Add(x2 >= 0, name="constraint4.2")
solver.Add(x3 >= 0, name="constraint4.3")
solver.Add(x4 >= 0, name="constraint4.4")

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001C754480C30> >

## Otimização

In [49]:
print("Number of variables =", solver.NumVariables())
print("Number of constraints =", solver.NumConstraints())

Number of variables = 4
Number of constraints = 7


In [50]:
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print("Solution:")
    print(f"Objective value = {solver.Objective().Value():0.1f}")
    print(f"x1 = {x1.solution_value():0.1f}")
    print(f"x2 = {x2.solution_value():0.1f}")
    print(f"x3 = {x3.solution_value():0.1f}")
    print(f"x4 = {x4.solution_value():0.1f}")
else:
    print("The problem does not have an optimal solution.")

Solving with Glop solver v9.10.4067
Solution:
Objective value = 93333.3
x1 = 0.0
x2 = 0.0
x3 = 6666.7
x4 = 6666.7


In [51]:
# Refs:
# https://developers.google.com/optimization/lp/lp_example